In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from utils import MultipleRegressionWithSoftmax, EmbeddingsDataset

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

In [ ]:
n_classes = 2
modality = 'music' # 'music', 'speech', or 'video'
which = 'openl3' # 'mfcc', 'msd' or 'openl3' for music, 'slow_fast' for video, 'hubert' for speech
voice = False 
drop_non_significant = True

fn_suffix = {
    'music': {
        'mfcc': '',
        'msd': '_backend', 
        'openl3': '_music', # '_music' or '_env'
    },
    'video': {
        'slow_fast': '_slow', # '_slow' or '_fast'
    },
    'speech': {
        'hubert': '_wave_encoder', # '_wave_encoder' or '_transformer'
    }
}

embedding_dimensions = {
    'video': {
        'slow_fast': 2048 if fn_suffix['video']['slow_fast']=='_slow' else 256,
    },
    'music': {
        'mfcc': 60,
        'msd': 256,
        'openl3': 512,
    },
    'speech': {
        'hubert': 1024 if fn_suffix['speech']['hubert']=='_transformer' else 512,
    }
}

## Load ground truth

In [ ]:
groundtruth_df = pd.read_csv("groundtruth_merged.csv")
groundtruth_df.set_index("stimulus_id", inplace=True)
groundtruth_df.head()

In [ ]:
emotions_and_mid_level = pd.read_csv("emotions_and_mid_level.csv")
emotions_and_mid_level.set_index("stimulus_id", inplace=True)

if drop_non_significant:
    # drop columns that are not significant based on the ANOVA test
    to_drop = ["Amusing", "Many/Few Instruments", "Wide/Narrow pitch variation", 
               "Repetitive/Non-repetitive", "Complex/Simple rhythm", "Fast tempo/Slow tempo"] 
    emotions_and_mid_level = emotions_and_mid_level.drop(columns=to_drop)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
emotions_and_mid_level.boxplot(ax=ax, rot=90)
plt.show()

In [ ]:
not_found = 0
for stimulus_id in groundtruth_df.index:
    if not os.path.exists(f"{modality}/embeddings_{which}/{stimulus_id}{fn_suffix[modality][which]}.npy"):
        print(f"Embedding for {stimulus_id} not found")
        not_found += 1

assert not_found == 0

## Load embeddings

In [ ]:
embedding_dim = embedding_dimensions[modality][which]

X = np.empty((groundtruth_df.shape[0], embedding_dim))
y_reg = np.empty((emotions_and_mid_level.shape[0], emotions_and_mid_level.shape[1]))

for i,stimulus_id in enumerate(groundtruth_df.index):
    embedding = np.load(f"{modality}/embeddings_{which}{'' if voice else '_novoice'}/" +
                        f"{stimulus_id}{fn_suffix[modality][which]}.npy")
    X[i] = embedding.mean(axis=0)
    y_reg[i] = emotions_and_mid_level.loc[stimulus_id].values

X.shape, y_reg.shape

In [ ]:
classes = ["Girls/women", "Boys/men"] if n_classes==2 else ["Girls/women", "Mixed", "Boys/men"]

y_cls = groundtruth_df.target.values

# convert to integers, and when the classes are not in 'classes' set them to -1
y_cls = groundtruth_df.target.values
y_cls = [classes.index(x) if x in classes else -1 for x in y_cls]
y_cls = np.array(y_cls)

pd.Series(y_cls).value_counts()

### partially annotated data will be handeled by the model, see utils.py

In [ ]:
params = {
    "input_dim": X.shape[1], 
    "n_regressions": y_reg.shape[1], 
    "output_dim": n_classes,
    }

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
f1s = []
r2s = []
pearsons = []
ps = []
for train_index, test_index in kf.split(X):

    test_index, val_index = train_test_split(test_index, test_size=0.5, random_state=42)
    
    X_train, X_test, X_val = X[train_index], X[test_index], X[val_index]
    y_reg_train, y_reg_test, y_reg_val = y_reg[train_index], y_reg[test_index], y_reg[val_index]
    y_cls_train, y_cls_test, y_cls_val = y_cls[train_index], y_cls[test_index], y_cls[val_index]

    train_dataset = EmbeddingsDataset(X_train, y_reg_train, y_cls_train)
    val_dataset = EmbeddingsDataset(X_val, y_reg_val, y_cls_val)
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=1)
 
    model = MultipleRegressionWithSoftmax(**params)
    
    checkpoint_callback = ModelCheckpoint(monitor='val_loss')
    trainer = pl.Trainer(max_epochs=100,
                         callbacks=[checkpoint_callback, EarlyStopping(monitor='val_loss', patience=30)],
                         enable_progress_bar = False,
                         accelerator='gpu',
                         devices=1)
    trainer.fit(model, train_loader, val_loader)
    
    # load best model
    model = model.load_from_checkpoint(checkpoint_callback.best_model_path, **params)
    
    # evaluate on test set
    model.eval()
    with torch.no_grad():
        y_reg_pred, out_cls = model(torch.from_numpy(X_test).float())
    
    skip_unlabelled = y_cls_test != -1
    y_cls_pred = torch.argmax(out_cls, dim=1).numpy()[skip_unlabelled]
    y_cls_test = y_cls_test[skip_unlabelled]

    accuracies.append(accuracy_score(y_cls_test, y_cls_pred))
    f1s.append(f1_score(y_cls_test, y_cls_pred, average='weighted'))

    r2_values = r2_score(y_reg_test, y_reg_pred, multioutput='raw_values')
    r2s.append(r2_values)

    r = [pearsonr(y_reg_test[:,i], y_reg_pred[:,i])[0] for i in range(y_reg_test.shape[1])]
    p = [pearsonr(y_reg_test[:,i], y_reg_pred[:,i])[1] for i in range(y_reg_test.shape[1])]
    pearsons.append(r)
    ps.append(p)


In [ ]:
#print(f"Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")
print(f"F1: {np.mean(f1s):.2f} ± {np.std(f1s):.2f}")

In [ ]:
# transpose to get r2s per feature
r2s = np.array(r2s).T

for i, r2 in enumerate(r2s):
    print(f"R2 for {emotions_and_mid_level.columns[i]}: {np.mean(r2):.2f} ± {np.std(r2):.2f}")

In [ ]:
# NB: compute standard deviation by row, and then average over all rows
print(f"Average R2 for all responses: {np.mean(r2s):.2f} ± {np.mean(np.std(r2s, axis=1)):.2f}")

In [ ]:
# transpose to get pearsons per feature
pearsons = np.array(pearsons).T
ps = np.array(ps).T

p2star = lambda p: ''.join(['*' for alpha in [0.001,0.01,0.05] if p<=alpha/len(ps[0])]) # Bonferroni correction

for i, r in enumerate(pearsons):
    print(f"Pearson's r for {emotions_and_mid_level.columns[i]}: {np.mean(r):.2f} ± {np.std(r):.2f} , significances {[p2star(p) for p in ps[i]]}")

In [ ]:
print(f"Average Pearson's r for all responses: {np.mean(pearsons):.2f} ± {np.mean(np.std(pearsons, axis=1)):.2f}")

### Let's see how the last trained model performs on the unlabeled data of the test set

In [ ]:
test_commercials = groundtruth_df.index[test_index]
no_actors_commercials = groundtruth_df[groundtruth_df.target == "There are no actors/presenters or you can never see their faces"].index
unseen_no_actors_commercials = no_actors_commercials.intersection(test_commercials)

In [ ]:
for stimulus_id in unseen_no_actors_commercials:
        # retrieve embedding
        embedding = np.load(f"{modality}/embeddings_{which}{'' if voice else '_novoice'}/" +
                            f"{stimulus_id}{fn_suffix[modality][which]}.npy")
        
        # predict
        with torch.no_grad():
            y_reg_pred, out_cls = model(torch.from_numpy(embedding.mean(axis=0)).float())

        y_cls_pred = torch.argmax(out_cls).numpy()

        # print results
        print(f"https://www.youtube.com/watch?v={stimulus_id} - predicted {classes[y_cls_pred]}")

        print("Regression:")
        for i, feature in enumerate(emotions_and_mid_level.columns):
            print(f"\t{feature}: predicted {y_reg_pred[i]:.2f}, actual {emotions_and_mid_level.loc[stimulus_id, feature]:.2f}")
        
        # # write to text file all the results
        # with open(f"results_{modality}_{which}.txt", "a") as f:
        #     f.write(f"https://www.youtube.com/watch?v={stimulus_id} - predicted {classes[y_cls_pred]}\n")
        #     f.write("Regression:\n")
        #     for i, feature in enumerate(emotions_and_mid_level.columns):
        #         f.write(f"\t{feature}: predicted {y_reg_pred[i]:.2f}, actual {emotions_and_mid_level.loc[stimulus_id, feature]:.2f}\n")
